In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path

In [9]:
data_dir = os.path.join(Path.cwd().parent, 'data', 'raw_data', "personal_budget_dataset.csv")

In [10]:
df = pd.read_csv(data_dir)

In [11]:
df.head()

,Month,Category,Budget,Actual Spending,Income,Spending Deviation
0,1,Groceries,402,322,4000,-80
1,1,Rent,1000,1002,4000,2
2,1,Utilities,201,222,4000,21
3,1,Entertainment,192,166,4000,-26
4,1,Transportation,114,101,4000,-13


In [12]:
df.isna().sum()

Month                 0
Category              0
Budget                0
Actual Spending       0
Income                0
Spending Deviation    0
dtype: int64

In [15]:
df.groupby(by=['Month', 'Actual Spending']).sum()

Category  Budget  Income  Spending Deviation
Month Actual Spending                                                    
1     101              Transportation     114    4000                 -13
      120                  Healthcare     121    4000                  -1
      166               Entertainment     192    4000                 -26
      222                   Utilities     201    4000                  21
      272                  Dining Out     256    4000                  16
...                               ...     ...     ...                 ...
24    263                  Dining Out     173    4000                  90
      285               Entertainment     198    4000                  87
      425                   Groceries     345    4000                  80
      661                     Savings     601    4000                  60
      994                        Rent    1000    4000                  -6

[192 rows x 4 columns]

In [19]:
df[(df["Category"] == "Transportation") & ( df["Month"] == 1 )]

,Month,Category,Budget,Actual Spending,Income,Spending Deviation
4,1,Transportation,114,101,4000,-13
